In [175]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as web
import seaborn as sns
from datetime import datetime
import yfinance as yf
yf.pdr_override()

In [176]:
# Lista de tickers
tickers = ["PETR4.SA", 'VALE3.SA', 'ITUB4.SA', 'BBAS3.SA', 'BBDC3.SA']

# Baixando os dados dos tickers
carteira_compra = yf.download(tickers, start="2002-01-01", end="2023-12-31")['Close']
carteira = yf.download(tickers, start="2002-01-01", end="2023-12-31")['Adj Close']

# Baixando os dados do índice Ibovespa
ibov = yf.download('^BVSP', start="2002-01-01", end="2023-12-31")['Adj Close']

[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed


In [177]:
resultados_vendacoberta = pd.read_json(r'C:\Users\cptfr\Desktop\TCC_B3_OPCOES\estrategias\resultados_vendacoberta.json')
resultados_vendacoberta['Data_Vencimento'] = pd.to_datetime(resultados_vendacoberta['Data_Vencimento'], format='%Y%m%d')
print(resultados_vendacoberta)

    Data_Entrada Data_Vencimento  Tipo     Ativo  ResultadosTotais
0       20041001      2004-11-16  CALL  PETR4.SA              0.56
1       20050919      2005-10-17  CALL  PETR4.SA              1.55
2       20070102      2007-02-12  CALL  PETR4.SA              0.03
3       20080519      2008-06-16  CALL  PETR4.SA              0.15
4       20090918      2009-10-19  CALL  BBDC3.SA              1.00
5       20130102      2013-02-18  CALL  BBAS3.SA              0.38
6       20131031      2013-12-16  CALL  BBAS3.SA              0.10
7       20140722      2014-09-15  CALL  BBAS3.SA              0.35
8       20140828      2014-10-20  CALL  BBAS3.SA              0.50
9       20161021      2016-12-19  CALL  PETR4.SA              1.00
10      20170921      2017-11-21  CALL  BBAS3.SA              0.55
11      20180122      2018-03-19  CALL  PETR4.SA              0.05
12      20180511      2018-06-18  CALL  VALE3.SA              0.23
13      20180201      2018-03-19  CALL  BBAS3.SA              

In [178]:
acoes_num = (10000/ carteira_compra.iloc[0]).astype(int)
acoes_num
#acoes_num['PETR4.SA']

BBAS3.SA    5665
BBDC3.SA    5200
ITUB4.SA    3431
PETR4.SA    1564
VALE3.SA    2307
Name: 2002-01-01 00:00:00, dtype: int32

In [179]:
carteira

,BBAS3.SA,BBDC3.SA,ITUB4.SA,PETR4.SA,VALE3.SA
Date,,,,,
2002-01-01,0.263994,1.026872,1.617728,1.571674,1.896705
2002-01-02,0.268730,1.026872,1.691356,1.574747,1.860231
2002-01-03,0.272968,1.034690,1.672880,1.585501,1.889411
2002-01-04,0.266736,1.030781,1.645396,1.575669,1.929533
2002-01-07,0.260753,1.034690,1.627102,1.610083,1.929533
...,...,...,...,...,...
2023-12-21,26.554209,14.914568,31.443792,33.934216,73.674408
2023-12-22,26.627577,14.951638,31.942442,34.260597,73.119240
2023-12-26,26.872135,15.011246,32.268486,34.810783,73.377686


In [180]:
def preencher_nans(df):
    # Preenche NaNs com o valor anterior (forward fill)
    df.ffill(inplace=True)
    # Para os casos onde o primeiro valor é NaN, preenche com o valor seguinte (backward fill)
    df.bfill(inplace=True)
    return df

In [181]:
inicio_2002 = '2002-01-01'
fim_2002 = '2002-12-31'

carteira_geral1 = carteira.loc[inicio_2002:fim_2002]
carteira_normalizada_2002 = (carteira_geral1 / carteira_geral1.iloc[0])*10000
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2002['saldo'] = carteira_normalizada_2002.sum(axis=1)
valores_ult_2002 = carteira_normalizada_2002['saldo'][-1] / (len(carteira_normalizada_2002.columns) -1)
print(carteira_normalizada_2002)
print(valores_ult_2002)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2002-01-01  10000.000000  10000.000000  10000.000000  10000.000000   
2002-01-02  10179.407239  10000.000000  10455.133429  10019.554529   
2002-01-03  10339.943765  10076.137435  10340.925989  10087.976798   
2002-01-04  10103.866814  10038.066976  10171.030771  10025.417626   
2002-01-07   9877.234251  10076.137435  10057.951516  10244.384207   
...                  ...           ...           ...           ...   
2002-12-25   9291.787774   8553.292954   9659.638471   9532.748318   
2002-12-26   9254.009095   8451.784113   9548.836217   9364.616990   
2002-12-27   9159.581024   9137.055858   9588.604148   9120.235816   
2002-12-30   9254.009095   9708.115646   9744.874206   9071.361250   
2002-12-31   9254.009095   9708.115646   9744.874206   9071.361250   

                VALE3.SA         saldo  
Date                                    
2002-01

In [182]:
inicio_2003 = '2003-01-01'
fim_2003 = '2003-12-31'

carteira_geral2 = carteira.loc[inicio_2003:fim_2003]
carteira_normalizada_2003 = (carteira_geral2 / carteira_geral2.iloc[0])*valores_ult_2002
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2003['saldo'] = carteira_normalizada_2003.sum(axis=1)
print(len(carteira_normalizada_2003.columns))
valores_ult_2003 = carteira_normalizada_2003['saldo'][-1] / (len(carteira_normalizada_2003.columns) -1)
print(carteira_normalizada_2003)
print(valores_ult_2003)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2003-01-01  11513.365461  11513.365461  11513.365461  11513.365461   
2003-01-02  11560.367764  11046.808745  11278.395283  11687.058080   
2003-01-03  11548.609814  10956.518859  11429.438150  11637.431480   
2003-01-06  11630.853662  11046.808745  11835.604406  12183.319268   
2003-01-07  11689.590740  11423.075575  11546.928898  11905.413773   
...                  ...           ...           ...           ...   
2003-12-25  28019.777214  14779.255589  19266.584709  18833.288503   
2003-12-26  27608.575533  14989.964574  18998.040646  18590.118066   
2003-12-29  27538.085421  15065.209128  19334.392411  18967.276761   
2003-12-30  28196.013729  15185.611705  19267.242901  18957.345858   
2003-12-31  28196.013729  15185.611705  19267.242901  18957.345858   

                VALE3.SA          saldo  
Date                                     
200

In [183]:
#PRIMEIRA ENTRADA
inicio_2004 = '2004-01-01'
fim_pre_2004 = '2004-11-16'

carteira_entrada_1 = carteira.loc[inicio_2004:fim_pre_2004]
carteira_normalizada_20041 = (carteira_entrada_1 / carteira_entrada_1.iloc[0])*valores_ult_2003
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20041['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[0]
carteira_normalizada_20041['saldo'] = carteira_normalizada_20041.sum(axis=1)
print(carteira_normalizada_20041)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2004-01-01  20114.275152  20114.275152  20114.275152  20114.275152   
2004-01-02  19862.846437  20034.535761  19606.171564  20851.450182   
2004-01-05  21497.125069  20792.056332  21025.371804  22062.523153   
2004-01-06  22293.328691  20253.827294  21025.371804  23220.939889   
2004-01-07  21748.561800  19894.983620  21270.678173  22976.095343   
...                  ...           ...           ...           ...   
2004-11-10  25980.934480  21081.117552  24389.461250  25063.865206   
2004-11-11  25821.690950  20782.095747  24221.258595  25156.021788   
2004-11-12  26064.747392  21226.133244  24319.361511  25221.835953   
2004-11-15  26064.747392  21226.133244  24319.361511  25221.835953   
2004-11-16  26819.017505  20931.603002  23898.848512  24912.931100   

                VALE3.SA          saldo  
Date                                     
2004-

In [184]:
inicio_2004_2 = '2004-11-16'
fim_2004 = '2004-12-31'

carteira_geral3 = carteira.loc[inicio_2004_2:fim_2004]
carteira_normalizada_2004 = carteira_geral3 / carteira_geral3.iloc[0]
for coluna in carteira_normalizada_2004.columns:
    if coluna != 'saldo':
        carteira_normalizada_2004[coluna] *= carteira_normalizada_20041[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2004['saldo'] = carteira_normalizada_2004.sum(axis=1)
print(len(carteira_normalizada_2004.columns))
valores_ult_2004 = carteira_normalizada_2004['saldo'][-1] / (len(carteira_normalizada_2004.columns) -1)
print(carteira_normalizada_2004)
print(valores_ult_2004)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2004-11-16  26819.017505  20931.603002  23898.848512  24912.931100   
2004-11-17  26567.606826  20916.660301  24039.038901  25114.857699   
2004-11-18  26408.353276  20707.340295  23681.596218  24803.783475   
2004-11-19  25897.129584  20483.090353  23758.690843  24694.637967   
2004-11-22  26860.934983  20527.923929  23688.605646  25019.349558   
2004-11-23  26081.503962  21079.640424  23217.645121  24585.488225   
2004-11-24  26282.656954  21081.117552  23584.171341  24449.051047   
2004-11-25  26827.419837  21377.137688  23555.444681  25758.826781   
2004-11-26  26827.419837  22345.989712  24039.038901  25600.560502   
2004-11-29  26735.214613  22275.724000  24381.024850  25172.157503   
2004-11-30  26693.323187  22277.213894  24809.265307  25349.523717   
2004-12-01  26483.783895  22724.251238  25300.566990  26113.552857   
2004-12-02  26902.

In [185]:
#ENTRADA 2
inicio_2005_1 = '2005-01-01'
fim_2005_1 = '2005-10-17'

carteira_entradaw = carteira.loc[inicio_2005_1:fim_2005_1]
carteira_entrada_2 = (carteira_entradaw / carteira_entradaw.iloc[0])*valores_ult_2004
#                                                      acessando primeira linha/primeira operação
carteira_entrada_2['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[1]
carteira_entrada_2['saldo'] = carteira_entrada_2.sum(axis=1)
print(carteira_entrada_2)


                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2005-01-03  26263.030787  26263.030787  26263.030787  26263.030787   
2005-01-04  25626.347977  25329.571215  25940.520824  25950.773345   
2005-01-05  25228.429784  24391.460244  25683.059460  25850.409131   
2005-01-06  24910.075059  23687.867361  25361.213974  25928.469706   
2005-01-07  24671.329472  24203.842341  25329.023582  26042.780725   
...                  ...           ...           ...           ...   
2005-10-10  33282.404149  48571.865365  34366.393087  36171.834698   
2005-10-11  34221.516431  50649.835340  34919.938941  36860.833275   
2005-10-13  32987.962992  48703.190174  33272.136798  36057.002323   
2005-10-14  32701.456869  47750.961946  33021.094814  35804.382780   
2005-10-17  32470.652025  49533.426429  34823.399487  39330.951515   

                VALE3.SA          saldo  
Date                                     
2005-

In [186]:
inicio_2005_2 = '2005-10-17'
fim_2005 = '2005-12-31'

carteira_geral4 = carteira.loc[inicio_2005_2:fim_2005]
carteira_normalizada_2005 = carteira_geral4 / carteira_geral4.iloc[0]
for coluna in carteira_normalizada_2005.columns:
    if coluna != 'saldo':
        carteira_normalizada_2005[coluna] *= carteira_entrada_2[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2005['saldo'] = carteira_normalizada_2005.sum(axis=1)
print(len(carteira_normalizada_2005.columns))
valores_ult_2005 = carteira_normalizada_2005['saldo'][-1] / (len(carteira_normalizada_2005.columns) -1)
print(carteira_normalizada_2005)
print(valores_ult_2005)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2005-10-17  32470.652025  49533.426429  34823.399487  39330.951515   
2005-10-18  32788.985818  49533.426429  33536.024219  37201.650065   
2005-10-19  32152.303008  47844.783771  32551.199569  37935.891309   
2005-10-20  30982.410619  46320.326088  31604.981351  36051.326182   
2005-10-21  31436.049119  47985.510071  32184.274845  37421.935349   
2005-10-24  31993.141820  48309.155674  33536.024219  39490.046954   
2005-10-25  32311.475614  48313.837111  32377.407180  39306.477421   
2005-10-26  31897.639780  48224.743924  33471.660130  39159.628250   
2005-10-27  32184.145902  46812.833811  32512.573102  38009.313588   
2005-10-28  31833.963506  47704.104671  32956.687317  38119.463146   
2005-10-31  32948.169840  49711.668586  34701.115732  39416.606230   
2005-11-01  33704.209269  52066.383980  34533.754409  40040.714053   
2005-11-03  33425.

In [187]:
inicio_2006 = '2006-01-01'
fim_2006 = '2006-12-31'

carteira_geral4 = carteira.loc[inicio_2006:fim_2006]
carteira_normalizada_2006 = (carteira_geral4 / carteira_geral4.iloc[0])*valores_ult_2005
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2006['saldo'] = carteira_normalizada_2006.sum(axis=1)
print(len(carteira_normalizada_2006.columns))
valores_ult_2006 = carteira_normalizada_2006['saldo'][-1] / (len(carteira_normalizada_2006.columns) -1)
print(carteira_normalizada_2006)
print(valores_ult_2006)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2006-01-02  41873.691148  41873.691148  41873.691148  41873.691148   
2006-01-03  42790.590295  43390.380965  43024.429117  43192.901748   
2006-01-04  48597.905677  43720.091814  44249.467630  43401.778360   
2006-01-05  50044.644455  43720.091814  43952.488830  43313.824724   
2006-01-06  52010.957467  44247.664162  45511.629456  44907.854698   
...                  ...           ...           ...           ...   
2006-12-21  64257.219472  51165.034831  56128.531052  54745.403951   
2006-12-22  63167.104553  50439.700519  55534.588857  54541.600174   
2006-12-26  63574.621781  51435.411361  56128.531052  55005.850294   
2006-12-27  64797.236802  52721.281865  57167.937595  56273.964953   
2006-12-28  65204.724798  54699.565062  57464.951058  56387.180651   

                VALE3.SA          saldo  
Date                                     
200

In [2]:
#ENTRADA 3 etapa 1
inicio_2007_1 = '2007-01-01'
fim_2007_1 = '2007-02-12'

carteira_entrada_2007 = carteira.loc[inicio_2007_1:fim_2007_1]
carteira_entrada_3 = (carteira_entrada_2007 / carteira_entrada_2007.iloc[0])*valores_ult_2006
#                                                      acessando primeira linha/primeira operação
carteira_entrada_3['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[2]
carteira_entrada_3['saldo'] = carteira_entrada_3.sum(axis=1)
print(carteira_entrada_3)

NameError: name 'carteira' is not defined

In [189]:
#ENTRADA 3 etapa 2
inicio_2007_2 = '2007-02-12'
fim_2007 = '2007-12-31'

carteira_geral_5 = carteira.loc[inicio_2007_2:fim_2007]
carteira_normalizada_2007 = carteira_geral_5 / carteira_geral_5.iloc[0]
for coluna in carteira_normalizada_2007.columns:
    if coluna != 'saldo':
        carteira_normalizada_2007[coluna] *= carteira_entrada_3[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2007['saldo'] = carteira_normalizada_2007.sum(axis=1)
print(len(carteira_normalizada_2007.columns))
valores_ult_2007 = carteira_normalizada_2007['saldo'][-1] / (len(carteira_normalizada_2007.columns) -1)
print(carteira_normalizada_2007)
print(valores_ult_2007)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA       PETR4.SA  \
Date                                                                  
2007-02-12  58126.274979  55991.048543  56572.157205   50952.418431   
2007-02-13  59006.965905  56060.042306  56498.165804   51885.125714   
2007-02-14  61120.604755  57281.296921  57792.328652   52000.286347   
2007-02-15  60169.475906  56929.404115  57385.598443   51009.986400   
2007-02-16  59015.797494  56577.549178  56794.008652   51044.532943   
...                  ...           ...           ...            ...   
2007-12-20  82169.355423  73026.436086  68847.924455   99433.003475   
2007-12-21  82169.355423  73053.993196  68788.809591  100604.293065   
2007-12-26  82697.844102  73757.778810  70252.986173  103290.098515   
2007-12-27  82644.980915  71756.877632  69143.698252  102154.232201   
2007-12-28  80319.918797  71066.851638  67295.018035  104591.597250   

                 VALE3.SA          saldo  
Date                           

In [190]:
#Entrada 4 etapa 1
inicio_2008_1 = '2008-01-01'
fim_2008_1 = '2008-06-16'

carteira_entrada_2008 = carteira.loc[inicio_2008_1:fim_2008_1]
carteira_entrada_4 = (carteira_entrada_2008 / carteira_entrada_2008.iloc[0])*valores_ult_2007
#                                                      acessando primeira linha/primeira operação
carteira_entrada_4['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[3]
carteira_entrada_4['saldo'] = carteira_entrada_4.sum(axis=1)
print(carteira_entrada_4)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2008-01-02  85653.853466  85653.853466  85653.853466  85653.853466   
2008-01-03  84360.373164  84601.273426  84039.988857  84275.533090   
2008-01-04  83383.182008  86382.359726  84630.402601  80140.510309   
2008-01-07  85366.428671  87237.178848  84630.402601  77787.494591   
2008-01-08  87119.704344  89214.315613  85417.622627  82060.342559   
...                  ...           ...           ...           ...   
2008-06-10  87090.986605  82759.392749  87336.607676  91159.531296   
2008-06-11  85366.428671  81957.198556  86106.549266  89970.515048   
2008-06-12  85107.804084  83962.655531  88074.703981  89534.509314   
2008-06-13  82779.532209  84470.676287  89083.366171  90961.376763   
2008-06-16  84504.117633  84604.401111  89304.711343  90997.774280   

                VALE3.SA          saldo  
Date                                     
2008-

In [191]:
#ENTRADA 4 etapa 2
inicio_2008_2 = '2008-06-16'
fim_2008 = '2008-12-31'

carteira_geral6 = carteira.loc[inicio_2008_2:fim_2008]
carteira_normalizada_2008 = carteira_geral6 / carteira_geral6.iloc[0]
for coluna in carteira_normalizada_2008.columns:
    if coluna != 'saldo':
        carteira_normalizada_2008[coluna] *= carteira_entrada_4[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2008['saldo'] = carteira_normalizada_2008.sum(axis=1)
print(len(carteira_normalizada_2008.columns))
valores_ult_2008 = carteira_normalizada_2008['saldo'][-1] / (len(carteira_normalizada_2008.columns) -1)
print(carteira_normalizada_2008)
print(valores_ult_2008)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2008-06-16  84504.117633  84604.401111  89304.711343  90997.774280   
2008-06-17  85510.154814  85968.169521  91641.924420  91891.894673   
2008-06-18  81917.294477  83695.238462  88443.680665  90203.066223   
2008-06-19  81917.294477  83668.473949  87951.626672  87222.790822   
2008-06-20  78784.275327  81261.997255  84384.426652  85692.905357   
...                  ...           ...           ...           ...   
2008-12-22  43258.041018  54090.771202  67556.716092  43988.880708   
2008-12-23  40240.034853  52154.172168  63989.408871  43710.721899   
2008-12-26  40240.034853  51723.834475  63718.812357  45300.203258   
2008-12-29  40240.034853  52396.270464  63718.812357  44827.642681   
2008-12-30  42194.595849  53768.053567  64210.907188  45464.623365   

                VALE3.SA          saldo  
Date                                     
200

In [192]:
#ENTRADA 5 etapa 1 
inicio_2009_1 = '2009-01-01'
fim_2009_1 = '2009-10-19'

carteira_entrada_2009 = carteira.loc[inicio_2009_1:fim_2009_1]
carteira_entrada_5 = (carteira_entrada_2009 / carteira_entrada_2009.iloc[0])*valores_ult_2008
#                                                      acessando primeira linha/primeira operação
carteira_entrada_5['BBDC3.SA'].iloc[-1] += acoes_num['BBDC3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[4]
carteira_entrada_5['saldo'] = carteira_entrada_5.sum(axis=1)
print(carteira_entrada_5)

                 BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                  
2009-01-02   49425.057902  49425.057902  49425.057902  49425.057902   
2009-01-05   50347.692163  49050.229525  49544.797338  50552.935663   
2009-01-06   53115.505658  49893.517120  51581.383367  51257.843628   
2009-01-07   51929.291621  49167.385116  48620.674086  49243.785132   
2009-01-08   50743.067079  49542.188666  48911.601251  51358.550723   
...                   ...           ...           ...           ...   
2009-10-13  108288.534459  69769.504149  67959.436415  74139.509964   
2009-10-14  111737.225609  72199.990566  69841.936799  74822.532334   
2009-10-15  110530.181080  72438.238499  69559.546472  75753.936582   
2009-10-16  107736.699756  70794.116382  68900.626238  75339.948254   
2009-10-19  108254.048073  76565.938376  69691.370291  77326.954640   

                VALE3.SA          saldo  
Date                              

In [193]:
#ENTRADA 5 etapa 2
inicio_2009_2 = '2009-10-19'
fim_2009 = '2009-12-31'

carteira_geral7 = carteira.loc[inicio_2009_2:fim_2009]
carteira_normalizada_2009 = carteira_geral7 / carteira_geral7.iloc[0]
for coluna in carteira_normalizada_2009.columns:
    if coluna != 'saldo':
        carteira_normalizada_2009[coluna] *= carteira_entrada_5[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2009['saldo'] = carteira_normalizada_2009.sum(axis=1)
print(len(carteira_normalizada_2009.columns))
valores_ult_2009 = carteira_normalizada_2009['saldo'][-1] / (len(carteira_normalizada_2009.columns) -1)
print(carteira_normalizada_2009)
print(valores_ult_2009)


6
                 BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                  
2009-10-19  108254.048073  76565.938376  69691.370291  77326.954640   
2009-10-20  104943.333981  74392.976943  68335.923708  75546.963265   
2009-10-21  105012.327763  74137.361728  67771.171462  75960.896002   
2009-10-22  107081.447912  75926.843263  69088.948009  76581.857647   
2009-10-23  104839.853813  75032.110106  68430.006469  75546.963265   
2009-10-26  103701.771552  74290.727813  68335.923708  75960.896002   
2009-10-27  100356.623597  72117.743550  66566.326899  74408.557904   
2009-10-28   93459.178270  69919.181785  62123.528268  70869.234267   
2009-10-29  100529.045024  73626.047588  66133.377166  74636.252874   
2009-10-30   97287.251182  71529.750172  63064.860135  72525.062498   
2009-11-03  100287.545779  72859.118233  66378.104560  73580.664635   
2009-11-04  103287.976935  72304.662509  68524.202865  73891.121136   
2009

In [3]:
inicio_2010 = '2010-01-01'
fim_2010 = '2010-12-31'

carteira_geral8 = carteira.loc[inicio_2010:fim_2010]
carteira_normalizada_2010 = (carteira_geral8 / carteira_geral8.iloc[0])*valores_ult_2009
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2010['saldo'] = carteira_normalizada_2010.sum(axis=1)
print(len(carteira_normalizada_2010.columns))
valores_ult_2010 = carteira_normalizada_2010['saldo'][-1] / (len(carteira_normalizada_2010.columns) -1)
print(carteira_normalizada_2010)
print(valores_ult_2010)

NameError: name 'carteira' is not defined

In [195]:
inicio_2011 = '2011-01-01'
fim_2011 = '2011-12-31'

carteira_geral9 = carteira.loc[inicio_2011:fim_2011]
carteira_normalizada_2011 = (carteira_geral9 / carteira_geral9.iloc[0])*valores_ult_2010
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2011['saldo'] = carteira_normalizada_2011.sum(axis=1)
print(len(carteira_normalizada_2011.columns))
valores_ult_2011 = carteira_normalizada_2011['saldo'][-1] / (len(carteira_normalizada_2011.columns) -1)
print(carteira_normalizada_2011)
print(valores_ult_2011)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2011-01-03  83130.907284  83130.907284  83130.907284  83130.907284   
2011-01-04  83475.111901  83396.868895  83801.321537  82823.035741   
2011-01-05  84666.465875  84035.898626  85058.372631  83808.266609   
2011-01-06  83925.166543  82022.861733  83047.106477  83469.586946   
2011-01-07  81939.616158  81383.839522  81014.886222  82299.587230   
...                  ...           ...           ...           ...   
2011-12-23  72982.000171  84297.363629  74455.698967  71056.106288   
2011-12-26  72860.068037  83766.079556  74047.940118  70896.210553   
2011-12-27  73348.020033  83932.115698  74259.999986  70992.158975   
2011-12-28  71579.113742  82836.537333  72571.818461  68529.800595   
2011-12-29  72280.609330  83965.328942  73567.415093  68721.677473   

                VALE3.SA          saldo  
Date                                     
201

In [196]:
inicio_2012 = '2012-01-01'
fim_2012 = '2012-12-31'

carteira_geral10 = carteira.loc[inicio_2012:fim_2012]
carteira_normalizada_2012 = (carteira_geral10 / carteira_geral10.iloc[0])*valores_ult_2011
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2012['saldo'] = carteira_normalizada_2012.sum(axis=1)
print(len(carteira_normalizada_2012.columns))
valores_ult_2012 = carteira_normalizada_2012['saldo'][-1] / (len(carteira_normalizada_2012.columns) -1)
print(carteira_normalizada_2012)
print(valores_ult_2012)

6
                BBAS3.SA       BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                  
2012-01-02  71851.606444   71851.606444  71851.606444  71851.606444   
2012-01-03  71526.221934   72148.533383  73646.272218  74709.642404   
2012-01-04  71171.273023   72148.533383  73730.754008  75243.046725   
2012-01-05  70431.743070   71608.860859  73181.801087  74376.253736   
2012-01-06  68656.876602   72688.199477  73730.754008  74442.937663   
...                  ...            ...           ...           ...   
2012-12-20  84904.645041  101277.553465  73335.486611  71265.887240   
2012-12-21  85583.833815  101944.060637  73746.473142  70047.088792   
2012-12-26  84904.645041  100334.295109  73663.588362  67914.183766   
2012-12-27  85108.405737   99834.350435  72440.933281  65713.578038   
2012-12-28  86942.285866   99804.948355  72855.707005  66085.991256   

                VALE3.SA          saldo  
Date                            

In [197]:
#ENTRADA 6 etapa 1
inicio_2013_1 = '2013-01-01'
fim_2013_1 = '2013-02-18'

carteira_entrada_2013 = carteira.loc[inicio_2013_1:fim_2013_1]
carteira_entrada_6 = (carteira_entrada_2013 / carteira_entrada_2013.iloc[0])*valores_ult_2012
#                                                      acessando primeira linha/primeira operação
carteira_entrada_6['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[5]
carteira_entrada_6['saldo'] = carteira_entrada_6.sum(axis=1)
print(carteira_entrada_6)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2013-01-02  80856.225203  80856.225203  80856.225203  80856.225203   
2013-01-03  82454.521136  85296.210947  83271.564145  83771.788749   
2013-01-04  81451.741581  85088.721888  82779.120204  83894.998152   
2013-01-07  81953.137609  83083.074927  81372.101359  82457.740845   
2013-01-08  82893.322350  82806.579030  81700.389155  80075.988718   
2013-01-09  83676.854426  84650.770040  83013.597491  80815.126187   
2013-01-10  82423.201854  84120.489994  81958.347645  81431.110596   
2013-01-11  81420.366047  83221.441272  82169.384551  81636.430386   
2013-01-14  84303.596335  84489.339280  83248.107435  80979.397043   
2013-01-15  83676.854426  83498.078236  82145.976828  81390.055403   
2013-01-16  84178.237953  84143.569524  82872.930714  81472.159530   
2013-01-17  84021.541538  84742.926942  84279.900572  80938.341850   
2013-01-18  82109.84

In [198]:
#ENTRADA 6 etapa 2
inicio_2013_2 = '2013-02-18'
fim_2013 = '2013-12-16'

carteira_geral11 = carteira.loc[inicio_2013_2:fim_2013]
carteira_normalizada_2013 = carteira_geral11 / carteira_geral11.iloc[0]
for coluna in carteira_normalizada_2013.columns:
    if coluna != 'saldo':
        carteira_normalizada_2013[coluna] *= carteira_entrada_6[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2013['saldo'] = carteira_normalizada_2013.sum(axis=1)
print(len(carteira_normalizada_2013.columns))
valores_ult_2013 = carteira_normalizada_2013['saldo'][-1] / (len(carteira_normalizada_2013.columns) -1)
print(carteira_normalizada_2013)
print(valores_ult_2013)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2013-02-18  77618.539773  84305.196770  83401.821140  73505.661552   
2013-02-19  79069.055371  82826.922873  81783.006087  74326.928189   
2013-02-20  78327.649447  81187.006614  80351.967179  72355.840681   
2013-02-21  81518.794048  81025.369292  79765.410641  71123.890643   
2013-02-22  82904.832858  81903.061514  82346.056928  70425.789586   
...                  ...           ...           ...           ...   
2013-12-10  94654.817895  80860.537815  83457.364473  72411.163149   
2013-12-11  92076.858478  79523.995711  81233.924766  70234.636906   
2013-12-12  94046.113301  79421.173154  81922.146096  70569.501544   
2013-12-13  94796.608974  79729.620674  82239.762844  72076.311032   
2013-12-16  94954.593176  79935.225484  82504.424417  72578.595468   

                VALE3.SA          saldo  
Date                                     
201

In [199]:
#Entrada 7 - Dupla entrada no mesmo ano
inicio_2013_22 = '2013-12-16'
fim_2013_22 = '2013-12-31'
carteira_geral12 = carteira.loc[inicio_2013_22:fim_2013_22]
carteira_normalizada_2013_22 = carteira_geral12 / carteira_geral12.iloc[0]
for coluna in carteira_normalizada_2013.columns:
    if coluna == 'BBAS3.SA':
        carteira_normalizada_2013_22[coluna] *= carteira_normalizada_2013[coluna].iloc[-1] + acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[6]
    elif coluna != 'saldo':
        carteira_normalizada_2013_22[coluna] *= carteira_normalizada_2013[coluna].iloc[-1]
carteira_normalizada_2013_22['saldo'] = carteira_normalizada_2013_22.sum(axis=1)
valores_ult_2013_22 = carteira_normalizada_2013_22['saldo'][-1] / (len(carteira_normalizada_2013_22.columns) -1)
print(valores_ult_2013_22)
print(carteira_normalizada_2013_22)


80860.88470981133
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2013-12-16  95521.093176  79935.225484  82504.424417  72578.595468   
2013-12-17  95600.569478  80449.262700  82610.285781  70318.349936   
2013-12-18  96156.845393  80218.003891  83113.302795  70485.775994   
2013-12-19  96951.504946  81400.284449  84966.097097  72201.880576   
2013-12-20  97348.834723  80860.537815  82954.445433  70778.771597   
2013-12-23  97428.336893  80603.514169  83843.212242  71406.606797   
2013-12-26  97150.189235  80584.167870  84429.352389  70820.609331   
2013-12-27  96474.705334  81285.365416  84189.559990  70862.490887   
2013-12-30  96951.504946  82973.440860  85441.754754  71490.313566   

                VALE3.SA          saldo  
Date                                     
2013-12-16  64954.258671  395493.597216  
2013-12-17  64727.601821  393706.069716  
2013-12-18  64916.467837  394890.395909  
2

In [200]:
#ENTRADA 8 etapa 1
inicio_2014_1 = '2014-01-01'
fim_2014_1 = '2014-09-15'

carteira_entrada_2014 = carteira.loc[inicio_2014_1:fim_2014_1]
carteira_entrada_8 = (carteira_entrada_2014 / carteira_entrada_2014.iloc[0])*valores_ult_2013_22
#                                                      acessando primeira linha/primeira operação
carteira_entrada_8['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[7]
carteira_entrada_8['saldo'] = carteira_entrada_8.sum(axis=1)
print(carteira_entrada_8)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2014-01-02   80860.884710   80860.884710   80860.884710   80860.884710   
2014-01-03   81703.186335   79046.891351   80706.939608   79654.024585   
2014-01-06   80490.290638   79582.511017   80835.189213   80233.309647   
2014-01-07   79108.885267   78077.631717   80219.503101   78012.648316   
2014-01-08   77525.357115   77848.074779   81476.555106   78157.474997   
...                   ...            ...            ...            ...   
2014-09-09  118263.622577  100028.142218  113106.659037  110567.863724   
2014-09-10  116399.208411   98834.909036  111885.510006  107839.680836   
2014-09-11  117240.040490   98860.783522  111710.937785  109178.046013   
2014-09-12  115608.395701   95773.806989  107698.464119  103670.249364   
2014-09-15  120727.791192   99068.323408  110431.679212  105935.135614   

                VALE3.SA          sal

In [201]:
#ENTRADA 8 etapa 2
#ERROOOOO
inicio_2014_2 = '2014-09-15'
fim_2014 = '2014-10-20'

carteira_geral13 = carteira.loc[inicio_2014_2:fim_2014]
carteira_normalizada_2014 = carteira_geral13 / carteira_geral13.iloc[0]
for coluna in carteira_normalizada_2014.columns:
    if coluna != 'saldo':
        carteira_normalizada_2014[coluna] *= carteira_entrada_8[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2014['saldo'] = carteira_normalizada_2014.sum(axis=1)
print(len(carteira_normalizada_2014.columns))
valores_ult_2014 = carteira_normalizada_2014['saldo'][-1] / (len(carteira_normalizada_2014.columns) -1)
print(carteira_normalizada_2014)
print(valores_ult_2014)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2014-09-15  120727.791192   99068.323408  110431.679212  105935.135614   
2014-09-16  126042.902162  100780.411251  113281.121247  110928.168650   
2014-09-17  126042.902162  100572.871365  113775.425603  113862.233878   
2014-09-18  124068.727963   98497.591194  112437.861040  109589.839575   
2014-09-19  121752.843609   97667.540450  109762.928363  107633.791276   
2014-09-22  115982.238030   97278.374727  108280.023153  105986.602589   
2014-09-23  114425.626572   96448.333873  107756.636524  103618.746288   
2014-09-24  113894.134429   97460.010454  108599.810294  104133.488240   
2014-09-25  110932.906580   95047.541023  105197.922930  102126.001847   
2014-09-26  113248.712886   98860.783522  109617.501279  107788.221082   
2014-09-29  103567.779975   91960.475471  101941.384071   95743.136660   
2014-09-30   96050.747501   90793.15

In [202]:
#Entrada 9 - Dupla entrada no mesmo ano
inicio_2014_22 = '2014-10-20'
fim_2014_22 = '2014-12-31'
carteira_geral13 = carteira.loc[inicio_2014_22:fim_2014_22]
carteira_normalizada_2014_22 = carteira_geral13 / carteira_geral13.iloc[0]
for coluna in carteira_normalizada_2014.columns:
    if coluna == 'BBAS3.SA':
        carteira_normalizada_2014_22[coluna] *= carteira_normalizada_2014[coluna].iloc[-1] + acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[8]
    elif coluna != 'saldo':
        carteira_normalizada_2014_22[coluna] *= carteira_normalizada_2014[coluna].iloc[-1]
carteira_normalizada_2014_22['saldo'] = carteira_normalizada_2014_22.sum(axis=1)
valores_ult_2014_22 = carteira_normalizada_2014_22['saldo'][-1] / (len(carteira_normalizada_2014_22.columns) -1)
print(valores_ult_2014_22)
print(carteira_normalizada_2014_22)

78572.75712238622
                 BBAS3.SA       BBDC3.SA       ITUB4.SA      PETR4.SA  \
Date                                                                    
2014-10-20  116422.934191   93257.483542  104139.174155  92242.819183   
2014-10-21  109341.037351   88066.394985   98903.146925  86992.407951   
2014-10-22  106811.824396   88066.394985   98263.226895  85499.641850   
2014-10-23   97083.955675   84380.744561   93812.523205  79837.400954   
2014-10-24  100196.894693   85367.031309   96430.564323  85551.101604   
2014-10-27   95021.692715   82667.776432   94307.016152  74741.412308   
2014-10-28  101753.352774   86327.433679   98612.253468  77366.617924   
2014-10-29   99068.446014   83809.745294   95761.467724  72167.659226   
2014-10-30  106033.629639   90298.554064  103499.191261  73711.906743   
2014-10-31  107901.406763   94685.031164  106844.438520  78653.480024   
2014-11-03  106500.528208   92297.160298  106509.807712  76440.075190   
2014-11-04  111870.296018   94215

In [203]:
inicio_2015 = '2015-01-01'
fim_2015 = '2015-12-31'

carteira_geral14 = carteira.loc[inicio_2015:fim_2015]
carteira_normalizada_2015 = (carteira_geral14 / carteira_geral14.iloc[0])*valores_ult_2014_22
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2015['saldo'] = carteira_normalizada_2015.sum(axis=1)
print(len(carteira_normalizada_2015.columns))
valores_ult_2015 = carteira_normalizada_2015['saldo'][-1] / (len(carteira_normalizada_2015.columns) -1)
print(carteira_normalizada_2015)
print(valores_ult_2015)

6
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2015-01-02  78572.757122  78572.757122  78572.757122  78572.757122   
2015-01-05  76942.318967  77604.251117  78967.612134  71857.154971   
2015-01-06  78017.707314  79673.773164  80245.063218  69506.673612   
2015-01-07  81451.982123  83191.813556  83148.248912  72780.536284   
2015-01-08  81729.524292  83375.762087  84448.898010  77481.487228   
...                  ...           ...           ...           ...   
2015-12-22  63602.221541  61354.559567  71411.294620  56998.827072   
2015-12-23  62966.196859  62084.263649  71142.819280  58174.070695   
2015-12-28  63517.409043  62551.278735  70874.350207  56243.322566   
2015-12-29  62754.200142  62463.712869  70928.077860  56159.384162   
2015-12-30  62499.787309  59836.762688  70686.435015  56243.322566   

                VALE3.SA          saldo  
Date                                     
201

In [204]:
#ENTRADA 10 etapa 1
inicio_2016_1 = '2016-01-01'
fim_2016_1 = '2016-12-19'

carteira_entrada_2016 = carteira.loc[inicio_2016_1:fim_2016_1]
carteira_entrada_10 = (carteira_entrada_2016 / carteira_entrada_2016.iloc[0])*valores_ult_2015
#                                                      acessando primeira linha/primeira operação
carteira_entrada_10['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[9]
carteira_entrada_10['saldo'] = carteira_entrada_10.sum(axis=1)
print(carteira_entrada_10)

                 BBAS3.SA      BBDC3.SA      ITUB4.SA       PETR4.SA  \
Date                                                                   
2016-01-04   59994.248437  59994.248437  59994.248437   59994.248437   
2016-01-05   60204.921195  60461.722993  60495.521957   58335.031875   
2016-01-06   60204.921195  59391.626774  60256.811794   55889.836778   
2016-01-07   58772.474851  57489.186815  59110.878263   54667.254542   
2016-01-08   58772.474851  57370.293071  59134.753734   54754.575282   
...                   ...           ...           ...            ...   
2016-12-13  119558.427240  94182.164916  89537.852864  134921.574553   
2016-12-14  116651.374514  92964.256744  89345.941300  128721.305882   
2016-12-15  121127.337422  93031.930952  88989.441022  129681.913648   
2016-12-16  121911.782710  94249.821619  87398.901319  128895.947363   
2016-12-19  119189.318608  92321.570585  85040.475880  126966.823760   

                 VALE3.SA          saldo  
Date                

In [205]:
#ENTRADA 10 etapa 2
inicio_2016_2 = '2016-12-19'
fim_2016 = '2016-12-30'

carteira_geral15 = carteira.loc[inicio_2016_2:fim_2016]
carteira_normalizada_2016 = carteira_geral15 / carteira_geral15.iloc[0]
for coluna in carteira_normalizada_2016.columns:
    if coluna != 'saldo':
        carteira_normalizada_2016[coluna] *= carteira_entrada_10[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2016['saldo'] = carteira_normalizada_2016.sum(axis=1)
print(len(carteira_normalizada_2016.columns))
valores_ult_2016 = carteira_normalizada_2016['saldo'][-1] / (len(carteira_normalizada_2016.columns) -1)
print(carteira_normalizada_2016)
print(valores_ult_2016)

6
                 BBAS3.SA      BBDC3.SA      ITUB4.SA       PETR4.SA  \
Date                                                                   
2016-12-19  119189.318608  92321.570585  85040.475880  126966.823760   
2016-12-20  121127.337422  93031.930952  87206.878369  126878.420173   
2016-12-21  121127.337422  91340.469625  87179.483117  126789.991782   
2016-12-22  119973.734490  91192.666424  87892.533797  123872.276545   
2016-12-23  123388.393679  93378.511820  90373.544496  125905.819489   
2016-12-26  125326.402690  95291.200937  91097.176794  127497.344499   
2016-12-27  126203.156994  95530.292547  91041.556468  127408.928510   
2016-12-28  127772.057374  97750.325146  93184.651552  130680.344909   
2016-12-29  129617.806381  99526.361728  94214.471319  131476.101213   

                 VALE3.SA          saldo  
Date                                      
2016-12-19  123879.651910  547397.840742  
2016-12-20  126400.032044  554644.598960  
2016-12-21  125924.525936  552361

In [206]:
#ENTRADA 11 etapa 1
inicio_2017_1 = '2017-01-01'
fim_2017_1 = '2017-11-21'

carteira_entrada_2017 = carteira.loc[inicio_2017_1:fim_2017_1]
carteira_entrada_11 = (carteira_entrada_2017 / carteira_entrada_2017.iloc[0])*valores_ult_2016
#                                                      acessando primeira linha/primeira operação
carteira_entrada_11['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[10]
carteira_entrada_11['saldo'] = carteira_entrada_11.sum(axis=1)
print(carteira_entrada_11)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2017-01-02  115390.978628  115390.978628  115390.978628  115390.978628   
2017-01-03  120670.344414  120067.196276  120277.828090  122002.751802   
2017-01-04  120041.862262  120027.170813  120793.965668  122002.751802   
2017-01-05  119748.564658  120708.157744  122445.914285  123970.570354   
2017-01-06  121047.424805  121629.687345  122858.915205  123262.142426   
...                   ...            ...            ...            ...   
2017-11-15  133880.717040  133827.388249  145543.748926  120822.093794   
2017-11-16  140224.701862  136770.567982  149207.330437  124442.789394   
2017-11-17  142587.308772  140159.763321  151270.215038  126095.765810   
2017-11-20  142587.308772  140159.763321  151270.215038  126095.765810   
2017-11-21  148153.186784  142567.939792  154293.459966  125151.228362   

                 VALE3.SA          sa

In [207]:
#ENTRADA 11 etapa 2
inicio_2017_2 = '2017-11-21'
fim_2017 = '2017-12-30'

carteira_geral17 = carteira.loc[inicio_2017_2:fim_2017]
carteira_normalizada_2017 = carteira_geral17 / carteira_geral17.iloc[0]
for coluna in carteira_normalizada_2017.columns:
    if coluna != 'saldo':
        carteira_normalizada_2017[coluna] *= carteira_entrada_11[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2017['saldo'] = carteira_normalizada_2017.sum(axis=1)
print(len(carteira_normalizada_2017.columns))
valores_ult_2017 = carteira_normalizada_2017['saldo'][-1] / (len(carteira_normalizada_2017.columns) -1)
print(carteira_normalizada_2017)
print(valores_ult_2017)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2017-11-21  148153.186784  142567.939792  154293.459966  125151.228362   
2017-11-22  148318.424104  140917.887251  152799.699196  126804.127492   
2017-11-23  148499.626910  139936.806819  152337.305012  127433.822804   
2017-11-24  147684.159733  138242.193759  150914.564267  126725.450081   
2017-11-27  145690.901596  139134.153151  151839.380167  124915.052597   
2017-11-28  147865.417090  138465.191302  152088.370122  124678.932036   
2017-11-29  141160.722352  136904.382924  148958.285417  120664.672726   
2017-11-30  135588.567879  135477.399749  146788.643637  121058.203314   
2017-12-01  139439.232055  136280.063677  146699.699970  122868.556634   
2017-12-04  142836.880126  137160.213455  148870.126426  121845.319694   
2017-12-05  140752.975126  135420.054885  147980.607161  120507.229577   
2017-12-06  145011.431988  137294.10

In [4]:
#ENTRADA 12, 14 E 15 etapa 1
inicio_2018_1 = '2018-01-01'
fim_2018_1 = '2018-03-19'

carteira_entrada_2018 = carteira.loc[inicio_2018_1:fim_2018_1]
carteira_entrada_12 = (carteira_entrada_2018 / carteira_entrada_2018.iloc[0])*valores_ult_2017
#                                                      acessando primeira linha/primeira operação
carteira_entrada_12['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[11]
carteira_entrada_12['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[13]
carteira_entrada_12['BBDC3.SA'].iloc[-1] += acoes_num['BBDC3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[14]
carteira_entrada_12['saldo'] = carteira_entrada_12.sum(axis=1)
print(carteira_entrada_12)

NameError: name 'carteira' is not defined

In [209]:
#ENTRADA 12, 14 E 15 etapa 2
inicio_2018_2 = '2018-03-19'
fim_2018 = '2018-06-18'

carteira_geral18 = carteira.loc[inicio_2018_2:fim_2018]
carteira_normalizada_2018 = carteira_geral18 / carteira_geral18.iloc[0]
for coluna in carteira_normalizada_2018.columns:
    if coluna != 'saldo':
        carteira_normalizada_2018[coluna] *= carteira_entrada_12[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2018['saldo'] = carteira_normalizada_2018.sum(axis=1)
print(len(carteira_normalizada_2018.columns))
valores_ult_2018 = carteira_normalizada_2018['saldo'][-1] / (len(carteira_normalizada_2018.columns) -1)
print(carteira_normalizada_2018)
print(valores_ult_2018)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2018-03-19  199825.158367  174319.788000  181893.702694  193280.164713   
2018-03-20  200597.468649  174031.931418  182474.249241  195404.215234   
2018-03-21  203927.823130  175615.432993  181748.505676  203623.006806   
2018-03-22  200259.477362  178254.397946  181167.865202  200575.584466   
2018-03-23  198859.775259  180413.580421  182038.805785  200852.606375   
...                   ...            ...            ...            ...   
2018-06-12  132830.139608  129055.625650  144866.544288  143052.515000   
2018-06-13  130418.645578  126782.186719  143449.809978  140369.096806   
2018-06-14  124512.886282  120279.204758  138364.197536  139721.373833   
2018-06-15  123036.455944  123345.642006  139708.272955  138518.480095   
2018-06-18  120674.140842  120014.868401  136874.871427  134169.510747   

                 VALE3.SA          

In [210]:
#ENTRADA 13 - Dupla Entrada mesmo ano
inicio_2018_22 = '2018-06-18'
fim_2018_22 = '2018-12-31'
carteira_geral13 = carteira.loc[inicio_2018_22:fim_2018_22]
carteira_normalizada_2018_22 = carteira_geral13 / carteira_geral13.iloc[0]
for coluna in carteira_normalizada_2018.columns:
    if coluna == 'VALE3.SA':
        carteira_normalizada_2018_22[coluna] *= carteira_normalizada_2018[coluna].iloc[-1] + acoes_num['VALE3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[12]
    elif coluna != 'saldo':
        carteira_normalizada_2018_22[coluna] *= carteira_normalizada_2018[coluna].iloc[-1]
carteira_normalizada_2018_22['saldo'] = carteira_normalizada_2018_22.sum(axis=1)
valores_ult_2018_22 = carteira_normalizada_2018_22['saldo'][-1] / (len(carteira_normalizada_2018_22.columns) -1)
print(valores_ult_2018_22)
print(carteira_normalizada_2018_22)

207045.08665689244
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2018-06-18  120674.140842  120014.868401  136874.871427  134169.510747   
2018-06-19  129139.101711  125301.842894  143050.246462  142682.357979   
2018-06-20  132879.365969  126094.894983  145702.034310  149992.285567   
2018-06-21  128007.142061  122182.525470  141815.139252  139721.373833   
2018-06-22  128450.065470  124773.180935  142904.955518  139813.913088   
...                   ...            ...            ...            ...   
2018-12-20  228022.917305  179047.407017  191994.080127  200141.023025   
2018-12-21  228074.600715  177764.645989  191107.292794  200699.781888   
2018-12-26  229574.766715  177130.922722  186950.385760  208855.231221   
2018-12-27  233402.640425  181602.938086  190109.645304  208758.883659   
2018-12-28  240489.642652  184602.428191  196760.811954  218488.756813   

                 V

In [211]:
#ENTRADA 16 etapa 1
inicio_2019_1 = '2019-01-01'
fim_2019_1 = '2019-03-18'

carteira_entrada_2019 = carteira.loc[inicio_2019_1:fim_2019_1]
carteira_entrada_16 = (carteira_entrada_2019 / carteira_entrada_2019.iloc[0])*valores_ult_2018_22
#                                                      acessando primeira linha/primeira operação
carteira_entrada_16['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[15]
carteira_entrada_16['saldo'] = carteira_entrada_16.sum(axis=1)
print(carteira_entrada_16)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2019-01-02  207045.086657  207045.086657  207045.086657  207045.086657   
2019-01-03  207897.133439  211372.493135  210458.670860  212122.289991   
2019-01-04  207897.133439  206739.632339  206933.370981  212724.648867   
2019-01-07  206533.896089  208476.955137  207324.931041  216080.793782   
2019-01-08  204190.716656  210793.317048  209955.043050  214789.942112   
2019-01-09  203636.930781  215194.461751  213144.654249  219264.717145   
2019-01-10  206619.039828  215831.486959  211689.818236  217371.552884   
2019-01-11  207471.086609  214962.859803  210402.664068  215048.135588   
2019-01-14  212285.122020  215194.461751  211969.581372  213843.371553   
2019-01-15  209814.250417  216005.239785  208667.956588  213671.242569   
2019-01-16  207811.942824  217163.432155  208052.382900  213585.178077   
2019-01-17  208578.736488  218032.1277

In [212]:
#ENTRADA 16 etapa 2
inicio_2019_2 = '2019-03-18'
fim_2019 = '2019-12-30'

carteira_geral19 = carteira.loc[inicio_2019_2:fim_2019]
carteira_normalizada_2019 = carteira_geral19 / carteira_geral19.iloc[0]
for coluna in carteira_normalizada_2019.columns:
    if coluna != 'saldo':
        carteira_normalizada_2019[coluna] *= carteira_entrada_16[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2019['saldo'] = carteira_normalizada_2019.sum(axis=1)
print(len(carteira_normalizada_2019.columns))
valores_ult_2019 = carteira_normalizada_2019['saldo'][-1] / (len(carteira_normalizada_2019.columns) -1)
print(carteira_normalizada_2019)
print(valores_ult_2019)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2019-03-18  237651.962500  232370.148912  212307.504366  247318.177661   
2019-03-19  232572.052303  227791.096474  207354.004199  251276.681452   
2019-03-20  227359.689130  223501.860371  202688.595064  249297.406415   
2019-03-21  222588.862123  216488.396376  199117.525611  245769.271367   
2019-03-22  210485.479378  210750.284059  193242.459133  232344.923127   
...                   ...            ...            ...            ...   
2019-12-20  243242.375757  244693.746741  214629.782137  265375.128263   
2019-12-23  250578.648367  249614.026809  216629.086503  267479.994049   
2019-12-26  253436.923716  252257.781872  219922.139647  271075.587552   
2019-12-27  252341.249399  250642.161388  219334.041245  271359.561034   
2019-12-30  251626.712061  250201.649690  218157.979853  268336.542863   

                 VALE3.SA         s

In [213]:
inicio_2020 = '2020-01-01'
fim_2020 = '2020-12-31'

carteira_geral20 = carteira.loc[inicio_2020:fim_2020]
carteira_normalizada_2020 = (carteira_geral20 / carteira_geral20.iloc[0])*valores_ult_2019
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2020['saldo'] = carteira_normalizada_2020.sum(axis=1)
print(len(carteira_normalizada_2020.columns))
valores_ult_2020 = carteira_normalizada_2020['saldo'][-1] / (len(carteira_normalizada_2020.columns) -1)
print(carteira_normalizada_2020)
print(valores_ult_2020)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2020-01-02  242009.559116  242009.559116  242009.559116  242009.559116   
2020-01-03  241604.709629  238233.207068  239464.217809  240038.801444   
2020-01-06  238410.885441  236866.839029  235900.555407  242876.642428   
2020-01-07  236611.556040  232835.877121  230427.722229  241930.729630   
2020-01-08  234452.333989  229283.248044  226673.233570  240432.969393   
...                   ...            ...            ...            ...   
2020-12-22  186468.471683  184914.709641  203906.329581  215053.814276   
2020-12-23  189536.526774  188866.830377  208408.566540  220335.553990   
2020-12-28  191630.566614  190462.928940  209909.204770  222148.693725   
2020-12-29  190510.482538  190443.866104  209452.507894  222858.179618   
2020-12-30  188952.117624  186739.944342  206385.825796  223410.047575   

                 VALE3.SA         s

In [214]:
#ENTRADA 17 etapa 1
inicio_2021_1 = '2021-01-01'
fim_2021_1 = '2021-02-22'

carteira_entrada_2021 = carteira.loc[inicio_2021_1:fim_2021_1]
carteira_entrada_17 = (carteira_entrada_2021 / carteira_entrada_2021.iloc[0])*valores_ult_2020
#                                                      acessando primeira linha/primeira operação
carteira_entrada_17['VALE3.SA'].iloc[-1] += acoes_num['VALE3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[16]
carteira_entrada_17['saldo'] = carteira_entrada_17.sum(axis=1)
print(carteira_entrada_17)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2021-01-04  242262.421331  242262.421331  242262.421331  242262.421331   
2021-01-05  239748.917269  241104.132915  240694.340826  251731.643242   
2021-01-06  245227.057560  247165.040278  247750.597543  252234.463510   
2021-01-07  254958.833734  253226.117080  257394.014684  259776.374941   
2021-01-08  256441.142008  251274.258770  257315.603622  260781.906424   
2021-01-11  252251.962011  247473.307206  251513.906310  258603.185810   
2021-01-12  254572.137774  247575.931063  252297.841006  256675.831709   
2021-01-13  242004.578101  242953.226169  248142.547298  244273.571540   
2021-01-14  241424.593204  250657.866112  255512.342708  246787.520206   
2021-01-15  233948.517224  241412.315124  245868.925566  235642.332518   
2021-01-18  231563.964299  242439.598567  244614.401347  235223.308358   
2021-01-19  227632.588169  237919.4892

In [215]:
#ENTRADA 17 etapa 2
inicio_2021_2 = '2021-02-22'
fim_2021 = '2021-07-16'

carteira_geral21 = carteira.loc[inicio_2021_2:fim_2021]
carteira_normalizada_2021 = carteira_geral21 / carteira_geral21.iloc[0]
for coluna in carteira_normalizada_2021.columns:
    if coluna != 'saldo':
        carteira_normalizada_2021[coluna] *= carteira_entrada_17[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2021['saldo'] = carteira_normalizada_2021.sum(axis=1)
print(len(carteira_normalizada_2021.columns))
valores_ult_2021 = carteira_normalizada_2021['saldo'][-1] / (len(carteira_normalizada_2021.columns) -1)
print(carteira_normalizada_2021)
print(valores_ult_2021)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2021-02-22  185805.421216  211253.693961  201238.974434  181592.065411   
2021-02-23  202232.028656  220926.260726  214125.780637  201620.000403   
2021-02-24  201434.566644  219485.643745  210982.617696  204469.155311   
2021-02-25  195785.633013  213826.152676  204381.996632  194329.499106   
2021-02-26  186414.986937  211871.018536  201908.432849  186368.607136   
...                   ...            ...            ...            ...   
2021-07-12  226554.846374  246412.277063  236571.329886  240194.869057   
2021-07-13  225581.965942  247225.897298  238005.617228  241667.407896   
2021-07-14  227110.859585  250364.086261  238324.345794  239675.189239   
2021-07-15  223636.126354  249201.787777  235455.771110  234564.629946   
2021-07-16  220925.771061  244901.187370  231631.098693  231099.916071   

                 VALE3.SA         s

In [216]:
#ENTRADA 18 e 19 - Dupla entrada mesmo ano
inicio_2021_22 = '2021-10-20'
fim_2021_22 = '2021-12-31'
carteira_geral21 = carteira.loc[inicio_2021_22:fim_2021_22]
carteira_normalizada_2021_22 = carteira_geral21 / carteira_geral21.iloc[0]
for coluna in carteira_normalizada_2021.columns:
    if coluna == 'ITUB4.SA':
        carteira_normalizada_2021_22[coluna] *= carteira_normalizada_2021[coluna].iloc[-1] + acoes_num['ITUB4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[17]
    elif coluna == 'BBAS3.SA':
        carteira_normalizada_2021_22[coluna] *= carteira_normalizada_2021[coluna].iloc[-1] + acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[18]
    elif coluna != 'saldo':
        carteira_normalizada_2021_22[coluna] *= carteira_normalizada_2021[coluna].iloc[-1]
carteira_normalizada_2021_22['saldo'] = carteira_normalizada_2021_22.sum(axis=1)
valores_ult_2021_22 = carteira_normalizada_2021_22['saldo'][-1] / (len(carteira_normalizada_2021_22.columns) -1)
print(valores_ult_2021_22)
print(carteira_normalizada_2021_22)

242612.0222775605
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2021-10-20  221095.721061  244901.187370  231974.198693  231099.916071   
2021-10-21  211316.493722  237896.580039  227240.019302  222585.711781   
2021-10-22  204867.820613  230098.794351  218514.307289  220396.310828   
2021-10-25  209119.721711  233799.412622  222784.363043  235478.660167   
2021-10-26  206922.969057  228380.665595  220370.842627  233208.209544   
2021-10-27  206710.301410  232213.456239  222041.818253  232640.611340   
2021-10-28  204867.820613  229966.637991  221577.653824  234829.935213   
2021-10-29  201962.466172  224680.063337  216379.310005  220153.046197   
2021-11-01  206568.600410  231949.047441  224996.904625  227045.505547   
2021-11-03  207915.050276  231383.543764  227133.553958  217720.399882   
2021-11-04  203025.436607  218815.478751  215149.880021  210827.959802   
2021-11-05  207560.7

In [217]:
#ENTRADA 20 E 21 - etapa 1
inicio_2022_1 = '2022-01-01'
fim_2022_1 = '2022-09-16'

carteira_entrada_2022 = carteira.loc[inicio_2022_1:fim_2022_1]
carteira_entrada20 = (carteira_entrada_2022 / carteira_entrada_2022.iloc[0])*valores_ult_2021
#                                                      acessando primeira linha/primeira operação
carteira_entrada20['ITUB4.SA'].iloc[-1] += acoes_num['ITUB4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[19]
carteira_entrada20['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[20]
carteira_entrada20['saldo'] = carteira_entrada20.sum(axis=1)
print(carteira_entrada20)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2022-01-03  250049.181276  250049.181276  250049.181276  250049.181276   
2022-01-04  250309.514462  251217.507162  257140.231363  250994.713447   
2022-01-05  246144.844924  249553.951301  252257.859514  241281.560651   
2022-01-06  248140.405766  252276.228686  257372.726459  241109.612649   
2022-01-07  248400.670527  253335.022330  263068.856323  242227.074638   
...                   ...            ...            ...            ...   
2022-09-12  392317.139150  275475.301809  320355.381213  404244.261316   
2022-09-13  388334.137198  271571.374369  317042.968131  393585.994581   
2022-09-14  388720.280605  270043.836819  315859.972451  399621.413095   
2022-09-15  384761.683464  268855.732025  316688.031660  398850.920208   
2022-09-16  380926.656067  266139.999530  315691.968137  395255.395837   

                 VALE3.SA         sal

In [218]:
#ENTRADA 20 E 21 - ETAPA 2
inicio_2022_2 = '2022-09-16'
fim_2022 = '2022-12-31'

carteira_geral22 = carteira.loc[inicio_2022_2:fim_2022]
carteira_normalizada_2022 = carteira_geral22 / carteira_geral22.iloc[0]
for coluna in carteira_normalizada_2022.columns:
    if coluna != 'saldo':
        carteira_normalizada_2022[coluna] *= carteira_entrada20[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2022['saldo'] = carteira_normalizada_2022.sum(axis=1)
print(len(carteira_normalizada_2022.columns))
valores_ult_2022 = carteira_normalizada_2022['saldo'][-1] / (len(carteira_normalizada_2022.columns) -1)
print(carteira_normalizada_2022)
print(valores_ult_2022)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2022-09-16  380926.656067  266139.999530  315691.968137  395255.395837   
2022-09-19  388467.846039  272589.780816  327524.572451  401547.608946   
2022-09-20  394365.510085  282603.966637  338410.448546  399236.203020   
2022-09-21  391561.635943  280736.960271  334860.740284  400263.526870   
2022-09-22  398716.119307  286338.141639  341605.319454  410151.264350   
...                   ...            ...            ...            ...   
2022-12-23  351615.519270  227514.243155  304535.695402  345108.877650   
2022-12-26  351117.245110  222235.167714  298490.969321  342635.924975   
2022-12-27  339456.528922  223597.442296  296798.377519  344971.515635   
2022-12-28  345535.966999  230409.283987  302843.128783  340712.565830   
2022-12-29  346134.046728  229387.578050  302238.646102  336591.087143   

                 VALE3.SA         s

In [219]:
#ENTRADA 22 E 23 etapa 1
inicio_2023_1 = '2023-01-01'
fim_2023_1 = '2023-07-21'

carteira_entrada_2023 = carteira.loc[inicio_2023_1:fim_2023_1]
carteira_entrada23 = (carteira_entrada_2023 / carteira_entrada_2023.iloc[0])*valores_ult_2022
#                                                      acessando primeira linha/primeira operação
carteira_entrada23['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[21]
carteira_entrada23['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[22]
carteira_entrada23['saldo'] = carteira_entrada23.sum(axis=1)
print(carteira_entrada23)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2023-01-02  305325.671272  305325.671272  305325.671272  305325.671272   
2023-01-03  299634.101623  290891.178814  298967.276091  297599.287879   
2023-01-04  303489.662741  293440.794979  299840.026512  307057.417068   
2023-01-05  317351.414608  302944.059056  309190.516065  318114.170355   
2023-01-06  321757.776182  311056.554337  314551.459949  316249.197068   
...                   ...            ...            ...            ...   
2023-07-17  489160.829722  361855.850792  368412.350770  463512.581341   
2023-07-18  490787.587514  360377.857189  361256.233311  460954.423092   
2023-07-19  482246.825116  359392.577137  357422.635106  465271.374645   
2023-07-20  487534.029858  361609.518525  364067.621709  465751.000665   
2023-07-21  493726.112225  371955.302186  370584.819110  474576.106129   

                 VALE3.SA         sal

In [220]:
#ENTRADA 22 E 23 etapa 2
inicio_2023_2 = '2023-07-21'
fim_2023 = '2023-12-31'

carteira_geral23 = carteira.loc[inicio_2023_2:fim_2023]
carteira_normalizada_2023 = carteira_geral23 / carteira_geral23.iloc[0]
for coluna in carteira_normalizada_2023.columns:
    if coluna != 'saldo':
        carteira_normalizada_2023[coluna] *= carteira_entrada23[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2023['saldo'] = carteira_normalizada_2023.sum(axis=1)
print(len(carteira_normalizada_2023.columns))
valores_ult_2023 = carteira_normalizada_2023['saldo'][-1] / (len(carteira_normalizada_2023.columns) -1)
print(carteira_normalizada_2023)
print(valores_ult_2023)

6
                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2023-07-21  493726.112225  371955.302186  370584.819110  474576.106129   
2023-07-24  484059.071160  365550.761274  364706.515815  484489.724149   
2023-07-25  481515.179469  363580.176661  365473.199123  495682.558504   
2023-07-26  485585.532495  366536.090342  367517.800404  495682.558504   
2023-07-27  475613.258112  358407.223560  359339.421233  469939.039487   
...                   ...            ...            ...            ...   
2023-12-21  586215.260135  383285.704177  427839.572101  627434.294064   
2023-12-22  587834.946269  384238.363768  434624.441685  633469.001447   
2023-12-26  593233.857941  385770.201963  439060.756468  643641.791877   
2023-12-27  592370.033758  388578.625087  442061.830341  644159.008175   
2023-12-28  598092.874235  390110.463281  443236.120718  642089.931385   

                 VALE3.SA         s